In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score,KFold,ShuffleSplit
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import r2_score,roc_auc_score,accuracy_score,confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import xgboost as xgb

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pd.set_option('max_columns',100)

In [49]:
train=pd.read_csv('/Users/shashank/Downloads/Societe2/train.csv')
test=pd.read_csv('/Users/shashank/Downloads/Societe2/test.csv')
sample=pd.read_csv('/Users/shashank/Downloads/Societe2/sample_submissions.csv')

In [50]:
# test2.drop(['transaction_id'],axis=1,inplace=True)

In [51]:
# ids=test['transaction_id']
# train.drop(['transaction_id'],axis=1,inplace=True)
# test.drop(['transaction_id'],axis=1,inplace=True)

In [52]:
train['cat_var_1'].fillna('gf',inplace=True)
test['cat_var_1'].fillna('gf',inplace=True)
train['cat_var_3'].fillna('qt',inplace=True)
test['cat_var_3'].fillna('qt',inplace=True)
test['cat_var_6'].fillna('zs',inplace=True)
train['cat_var_8'].fillna('dn',inplace=True)
test['cat_var_8'].fillna('dn',inplace=True)

In [53]:
df=pd.concat([train,test])
for i in range(1,19):
    if i not in [1,3,6,8]:
        dic=dict(df['cat_var_%d'%i].value_counts())
        df['var_%d_counts'%i]=df['cat_var_%d'%i].apply(lambda x: dic[x])
        df.drop(['cat_var_%d'%i],axis=1,inplace=True)

In [54]:
train=df.iloc[:len(train),:]
test=df.iloc[len(train):,:]

In [71]:
print len(train),len(test)

348978 523466


In [28]:
li=[]
for index,row in test.iterrows():
    if (row['cat_var_1'] in ['jk', 'gz', 'yw']) or (row['cat_var_3'] in ['ex', 'ye', 'hr', 'ap']) or (row['cat_var_8'] in ['da', 'db', 'mh']):
        li.append(True)
    else: li.append(False)

In [55]:
test2=test[[not i for i in li]]

In [56]:
test2

,cat_var_1,cat_var_19,cat_var_20,cat_var_21,cat_var_22,cat_var_23,cat_var_24,cat_var_25,cat_var_26,cat_var_27,cat_var_28,cat_var_29,cat_var_3,cat_var_30,cat_var_31,cat_var_32,cat_var_33,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42,cat_var_6,cat_var_8,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,target,transaction_id,var_2_counts,var_4_counts,var_5_counts,var_7_counts,var_9_counts,var_10_counts,var_11_counts,var_12_counts,var_13_counts,var_14_counts,var_15_counts,var_16_counts,var_17_counts,var_18_counts
1,pu,0,1,1,0,0,1,0,0,0,0,0,qt,0,0,0,0,0,0,0,0,0,0,0,0,0,oi,jo,6.578947e-08,0.086391,0.0,7.500000e-07,1.315789e-07,9.868421e-08,3.186300e-08,NaN,id_6,582056,289199,436279,871363,183011,24118,79011,72522,53281,67241,590455,71439,634373,374778
4,gf,0,1,0,1,0,1,0,0,0,0,0,qt,0,0,0,0,0,0,0,0,0,0,0,0,0,zs,dn,1.144737e-07,0.089563,0.0,3.550000e-07,4.671053e-08,4.407895e-08,1.452953e-07,NaN,id_15,582056,583245,436279,871363,175016,80714,205398,357150,332353,620874,590455,801005,634373,497666
5,aq,1,0,0,0,1,1,0,0,0,0,0,fm,0,0,0,0,0,0,0,0,0,0,0,0,0,od,qd,8.572368e-07,0.582214,0.0,7.680000e-06,2.631579e-07,1.485526e-06,2.048791e-07,NaN,id_19,582056,289199,436279,871363,204002,30008,196216,64132,4488,21100,590455,801005,634373,497666
6,xd,1,0,1,0,0,1,0,0,0,0,0,qt,0,0,0,0,0,0,0,0,0,0,0,0,0,sn,al,1.578947e-07,0.293751,0.0,5.865000e-06,6.355263e-07,1.578947e-07,7.647119e-08,NaN,id_20,582056,289199,436165,871363,183011,25230,196216,64132,647,17361,590455,801005,634373,497666
8,ph,0,1,1,0,0,1,0,0,0,0,0,rl,0,0,0,0,0,0,0,0,0,0,0,0,0,ej,ei,9.361842e-07,0.494026,0.0,2.890000e-06,3.407895e-07,3.730263e-07,7.647119e-09,NaN,id_25,582056,289199,436165,871363,168244,38575,190308,357150,491,14237,590455,71439,634373,374778
9,gf,0,1,0,1,0,1,0,0,0,0,0,ip,0,0,0,0,0,0,0,0,0,0,0,0,0,zs,dn,5.263158e-09,0.078355,0.0,3.550000e-07,4.671053e-08,4.407895e-08,4.938765e-08,NaN,id_28,582056,583245,436279,871363,204002,77365,201511,357150,332353,620874,590455,801005,634373,497666
10,gf,0,1,0,1,0,1,0,0,0,0,0,cg,0,0,0,0,0,0,0,0,0,0,0,0,0,zs,dn,5.921053e-09,0.089669,0.0,3.550000e-07,4.671053e-08,4.407895e-08,1.911780e-09,NaN,id_31,582056,583245,436279,871363,183011,80714,205398,357150,332353,620874,590455,801005,634373,497666
11,me,1,0,1,0,0,1,0,0,0,0,0,qt,0,0,0,0,0,0,0,0,0,0,0,0,0,cg,pk,1.657895e-07,0.064502,0.0,1.169000e-05,8.072368e-07,1.394079e-06,6.882407e-08,NaN,id_32,582056,289199,436165,871363,168244,25230,196216,64132,3392,17361,590455,801005,634373,497666
12,so,0,1,0,1,0,1,0,0,0,0,0,es,0,0,0,0,0,0,0,0,0,0,0,0,0,le,zy,3.770395e-06,0.575553,0.0,1.573500e-05,1.879605e-06,4.407895e-08,2.578672e-06,NaN,id_37,582056,289199,436279,871363,175016,86866,196216,72522,1984,14237,590455,801005,634373,497666
13,gf,0,1,0,1,0,1,0,0,0,0,0,qt,0,0,0,0,0,0,0,0,0,0,0,0,0,zs,dn,9.210526e-09,0.089563,0.0,3.550000e-07,4.671053e-08,4.407895e-08,2.230410e-09,NaN,id_41,582056,583245,436279,871363,175016,80714,205398,357150,332353,620874,590455,801005,634373,497666


In [58]:
TN= float(len(train[train['target']==0]))
TP= float(len(train[train['target']==1]))

In [31]:
cat_1={}
for i in train.cat_var_1.unique():
    pi=float(len(train[train['cat_var_1']==i][train['target']==1]))
    ni=float(len(train[train['cat_var_1']==i][train['target']==0]))
    if ni==0.0: ni=0.0000000001
    if pi==0.0: pi=0.0000000001
    cat_1[i]=(np.log((pi/TP)/(ni/TN)))

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [32]:
cat_3={}
for i in train.cat_var_3.unique():
    pi=float(len(train[train['cat_var_3']==i][train['target']==1]))
    ni=float(len(train[train['cat_var_3']==i][train['target']==0]))
    if ni==0.0: ni=0.0000000001
    if pi==0.0: pi=0.0000000001
    cat_3[i]=(np.log((pi/TP)/(ni/TN)))

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [33]:
cat_6={}
for i in train.cat_var_6.unique():
    pi=float(len(train[train['cat_var_6']==i][train['target']==1]))
    ni=float(len(train[train['cat_var_6']==i][train['target']==0]))
    if ni==0.0: ni=0.0000000001
    if pi==0.0: pi=0.0000000001
    cat_6[i]=(np.log((pi/TP)/(ni/TN)))

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [34]:
cat_8={}
for i in train.cat_var_8.unique():
    pi=float(len(train[train['cat_var_8']==i][train['target']==1]))
    ni=float(len(train[train['cat_var_8']==i][train['target']==0]))
    if ni==0.0: ni=0.0000000001
    if pi==0.0: pi=0.0000000001
    cat_8[i]=(np.log((pi/TP)/(ni/TN)))

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [59]:
train['cat_var_1']=train['cat_var_1'].apply(lambda x: cat_1[x])
test2['cat_var_1']=test2['cat_var_1'].apply(lambda x: cat_1[x])

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [60]:
train['cat_var_3']=train['cat_var_3'].apply(lambda x: cat_3[x])
test2['cat_var_3']=test2['cat_var_3'].apply(lambda x: cat_3[x])
train['cat_var_6']=train['cat_var_6'].apply(lambda x: cat_6[x])
test2['cat_var_6']=test2['cat_var_6'].apply(lambda x: cat_6[x])
train['cat_var_8']=train['cat_var_8'].apply(lambda x: cat_8[x])
test2['cat_var_8']=test2['cat_var_8'].apply(lambda x: cat_8[x])

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [64]:
trainX=train.copy(deep=True)
testX=test2.copy(deep=True)

In [67]:
predictors=[x for x in trainX.columns if x not in ['target','transaction_id']]
target=['target']

In [69]:
def run(X,y,test,params):
    dtrain=xgb.DMatrix(X,label=y)
    dtest=xgb.DMatrix(test)
    bst=xgb.train(params,dtrain,89)
    preds=bst.predict(dtest)
    return bst,preds

params={
    'objective':'binary:logistic',
    'eta':0.1,
    'max_depth':9,
    'min_child_weight':7,
    'gamma':0.1,
    'subsample':0.9,
    'colsample_bytree':0.7,
    'scale_pos_weight':1,
    'reg_alpha':1e-05
}

In [70]:
bst,xgpreds=run(trainX[predictors].values,trainX[target].values,testX[predictors].values,params)

In [81]:
log=pd.DataFrame({'transaction_id':test2['transaction_id'],'target':xgpreds,},columns=['transaction_id','target'])

In [83]:
log.to_csv('/Users/shashank/Downloads/log1.csv')